# Libraries/Imports

In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install datasets
%pip install kagglehub
%pip install transformers
%pip install accelerate
%pip install latex2sympy
%pip install --upgrade torch torchvision torchaudio
%pip install --upgrade torchtext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [ ]:
def print_rows(dataset, start=0, end=10, split="", feature=""):
  if split == "":
    for split in dataset:
      print(f"Entries {start+1} - {end} of the {split} data:")
      for i in range(start, end):
        if feature:  # If feature is provided
          print(f"{feature}: {dataset[split][i][feature]}")  # Print the specified feature
        else:
          print(dataset[split][i])  # Print the entire row if no feature is specified
      print("-" * 20)
  else:
    print(f"Entries {start+1} - {end} of the {split} data:")
    for i in range(start, end):
      if feature:  # If feature is provided
        print(f"{feature}: {dataset[split][i][feature]}")  # Print the specified feature
      else:
        print(dataset[split][i])  # Print the entire row if no feature is specified
    print("-" * 20)

# Preprocessing

In [ ]:
# Preprocess COT dataset
from datasets import load_dataset

cot_ds = load_dataset("AI-MO/NuminaMath-CoT")

cot_ds['train'] = cot_ds['train'].remove_columns(['messages'])
cot_ds['test'] = cot_ds['test'].remove_columns(['messages'])
cot_ds['train'] = cot_ds['train'].remove_columns(['source'])
cot_ds['test'] = cot_ds['test'].remove_columns(['source'])

# Remove chinese characters from COT dataset
import re

def contains_chinese(text):
    # match Chinese characters
    pattern = re.compile(r'[\u4e00-\u9fff\u2e80-\u2eff\u31c0-\u31ef\uff00-\uffef]')
    return bool(pattern.search(text))

def filter_entries(dataset, fields):
    # Filter out entries that contain Chinese characters
    filtered_dataset = dataset.filter(lambda example: not any(contains_chinese(example[field]) for field in fields))
    return filtered_dataset

# remove entries with Chinese characters
fields_to_check = ['problem', 'solution']
cot_ds['train'] = filter_entries(cot_ds['train'], fields_to_check)
cot_ds['test'] = filter_entries(cot_ds['test'], fields_to_check)
print(cot_ds)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/247M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/859494 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/859494 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution'],
        num_rows: 850151
    })
    test: Dataset({
        features: ['problem', 'solution'],
        num_rows: 100
    })
})


In [ ]:
# Preprocess MATH dataset (load all json files into into Dataset object)

import kagglehub

# Download latest version
path = kagglehub.dataset_download("mathurinache/math-dataset")

print("Path to dataset files:", path)

import os
from datasets import Dataset, DatasetDict

def load_json_files(data_dir):
    """Loads JSON files from a directory into a Dataset."""
    all_data = []
    problems = 0
    for subdir in os.listdir(data_dir):
      subdir_path = os.path.join(data_dir, subdir)
      for filename in os.listdir(subdir_path):
        if filename.endswith(".json"):
          problems += 1
          filepath = os.path.join(subdir_path, filename)
          with open(filepath, "r") as f:
            all_data.append(json.load(f))
    # Create a Pandas DataFrame to easily convert into a Dataset\
    print(f"Loaded {problems} problems.")
    return all_data

# Assuming 'path' is from kagglehub.dataset_download
math_dir = os.path.join(path, "MATH")
train_dir = os.path.join(math_dir, "train")
test_dir = os.path.join(math_dir, "test")

train_data = load_json_files(train_dir)
test_data = load_json_files(test_dir)

# Convert the train and test data into Dataset objects
train_dataset = Dataset.from_dict({
    "problem": [item["problem"] for item in train_data],
    # "level": [item["level"] for item in train_data],
    # "type": [item["type"] for item in train_data],
    "solution": [item["solution"] for item in train_data]
})

test_dataset = Dataset.from_dict({
    "problem": [item["problem"] for item in test_data],
    # "level": [item["level"] for item in test_data],
    # "type": [item["type"] for item in test_data],
    "solution": [item["solution"] for item in test_data]
})

math_ds = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(math_ds)


100%|██████████| 7.07M/7.07M [00:00<00:00, 47.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mathurinache/math-dataset/versions/1
Loaded 7500 problems.
Loaded 5000 problems.
DatasetDict({
    train: Dataset({
        features: ['problem', 'solution'],
        num_rows: 7500
    })
    test: Dataset({
        features: ['problem', 'solution'],
        num_rows: 5000
    })
})


In [ ]:
# Concatenate and split datasets
from datasets import concatenate_datasets

# Make validation dataset
train_valid_split = cot_ds['train'].train_test_split(test_size=0.1)
cot_ds['train'] = train_valid_split['train']
cot_ds['test'] = train_valid_split['test']

train_valid_split = cot_ds['train'].train_test_split(test_size=0.12)
cot_ds['train'] = train_valid_split['train']
cot_ds['validation'] = train_valid_split['test']

# Add MATH dataset as test dataset
merged_math = concatenate_datasets([math_ds['train'], math_ds['test']])
cot_ds['test'] = concatenate_datasets([cot_ds['test'], merged_math])

ds = cot_ds
print(ds)

del cot_ds
del math_ds

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution'],
        num_rows: 673318
    })
    test: Dataset({
        features: ['problem', 'solution'],
        num_rows: 97516
    })
    validation: Dataset({
        features: ['problem', 'solution'],
        num_rows: 91817
    })
})


In [ ]:
print("Split")
print("train:", len(ds['train']) / ( len(ds['train']) + len(ds['validation']) + len(ds['test']) ))
print("test:", len(ds['test']) / ( len(ds['train']) + len(ds['validation']) + len(ds['test']) ))
print("validation:", len(ds['validation']) / ( len(ds['train']) + len(ds['validation']) + len(ds['test']) ))

Split
train: 0.7805219028320839
test: 0.11304223840232029
validation: 0.10643585876559582


# Embedding

In [ ]:
ds200 = DatasetDict({
    split: dataset.select(range(200))
    for split, dataset in ds.items()
})
print(ds200)

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution'],
        num_rows: 200
    })
    test: Dataset({
        features: ['problem', 'solution'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['problem', 'solution'],
        num_rows: 200
    })
})


In [ ]:
# Generate Embeddings

from transformers import AutoTokenizer, AutoModel
import torch

# Load MathBERT
model_name = "tbs17/MathBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to compute embeddings
def compute_embeddings(batch):
    # Tokenize the problem text
    inputs = tokenizer(batch["problem"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Move input tensors to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token embedding (typically the first token)
    cls_embeddings = outputs.last_hidden_state.squeeze(0).cpu().numpy()

    # Return embeddings
    return {"embeddings": cls_embeddings}

# Apply the function to each split of the dataset
def process_dataset(dataset_dict):
    for split in dataset_dict:
        dataset_dict[split] = dataset_dict[split].map(compute_embeddings, batched=True, batch_size=64)
        dataset_dict[split].set_format(type="torch", columns=["embeddings"], output_all_columns=True)
    return dataset_dict


# Add embeddings to the dataset
ds200 = process_dataset(ds200)

# Save embeddings or access them
torch.save(model.state_dict(), 'mathbert_weights.pth')
# model.load_state_dict(torch.load('mathbert_weights.pth'))

print(ds200)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/441M [00:00<?, ?B/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['problem', 'solution', 'embeddings'],
        num_rows: 200
    })
    test: Dataset({
        features: ['problem', 'solution', 'embeddings'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['problem', 'solution', 'embeddings'],
        num_rows: 200
    })
})


In [ ]:
batch_embeddings = []
for i in range(len(ds200['train'])):
  p= torch.tensor(ds200['train'][i]["embeddings"]).unsqueeze(0)
  batch_embeddings.append(p)

print(len(batch_embeddings))
batch_embeddings = torch.cat(batch_embeddings, dim=0)
print(batch_embeddings.shape)

<ipython-input-10-14517263a1ad>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  p= torch.tensor(ds200['train'][i]["embeddings"]).unsqueeze(0)


200
torch.Size([200, 128, 768])


# ENCODER


In [ ]:
print(ds200['train'][0])

{'embeddings': tensor([[-1.1499, -1.1960,  1.4161,  ..., -1.5777,  0.6454,  0.7011],
        [-0.5345,  1.3915,  0.7776,  ..., -0.5360, -0.0076,  2.3830],
        [ 0.1135,  1.5901,  0.8758,  ..., -0.8767,  0.8682,  2.3415],
        ...,
        [-0.6988, -1.7332,  1.1532,  ..., -0.2709,  0.4264,  0.5452],
        [-1.2455, -0.9665,  1.0044,  ...,  0.2618, -0.1053,  0.0486],
        [-0.5270, -0.1318,  0.7925,  ...,  0.0084, -0.2222,  0.4713]]), 'problem': 'Given a stick 7 cm long, a stick 12 cm long, and a third stick \\( n \\) cm long are to form a triangle. Determine the sum of all possible whole number values of \\( n \\).', 'solution': 'Using the Triangle Inequality:\n1. \\( n + 7 > 12 \\) leads to \\( n > 5 \\).\n2. \\( n + 12 > 7 \\) leads to \\( n > -5 \\) (but \\( n \\) must be positive so this condition is automatically satisfied).\n3. \\( 7 + 12 > n \\) leads to \\( n < 19 \\).\n\nConsidering positive whole numbers, \\( n \\) must be greater than 5 and less than 19. Therefor

In [ ]:
train_embeddings = ds200["train"]["embeddings"]
print(train_embeddings[1])

print(len(train_embeddings[1]))


tensor([[-1.0516, -0.8992,  2.2650,  ..., -1.2516,  0.8548,  0.2484],
        [ 0.6413, -0.0655,  0.5192,  ...,  0.2561, -0.4663, -1.1526],
        [ 0.3552,  1.9795,  0.5843,  ...,  0.7809,  0.2402, -0.0583],
        ...,
        [ 0.4893, -0.5208,  0.7787,  ..., -0.3071, -0.1800,  0.7336],
        [ 0.7425, -1.4363,  0.1554,  ..., -0.3766, -0.7310,  0.8924],
        [ 0.5438, -2.0659, -0.0051,  ..., -0.3236, -0.1524,  0.9549]])
128


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from typing import Union, Tuple


In [ ]:
class CustomConfig:
    def __init__(
        self,
        d_model=768,      # Hidden size
        num_heads=8,       # Number of attention heads
        d_ff= 3072,         # Feedforward network size
        num_encoder_layers=6,  # Number of encoder layers
        dropout_rate=0.1,      # Dropout rate
        attention_dropout_rate=0.1,  # Attention dropout rate
        layer_norm_eps=1e-6,   # Layer norm epsilon for stability
    ):
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_ff = d_ff
        self.num_encoder_layers = num_encoder_layers
        self.dropout_rate = dropout_rate
        self.attention_dropout_rate = attention_dropout_rate
        self.layer_norm_eps = layer_norm_eps

        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"


In [ ]:
train_embedding = torch.tensor(ds200['train'][1]["embeddings"])
train_embedding = train_embedding.unsqueeze(0)

print(train_embedding)
print(train_embedding.shape)
# need to convert embeddings into tensor-> mb add nn.embedding as the first layer

tensor([[[-1.0516, -0.8992,  2.2650,  ..., -1.2516,  0.8548,  0.2484],
         [ 0.6413, -0.0655,  0.5192,  ...,  0.2561, -0.4663, -1.1526],
         [ 0.3552,  1.9795,  0.5843,  ...,  0.7809,  0.2402, -0.0583],
         ...,
         [ 0.4893, -0.5208,  0.7787,  ..., -0.3071, -0.1800,  0.7336],
         [ 0.7425, -1.4363,  0.1554,  ..., -0.3766, -0.7310,  0.8924],
         [ 0.5438, -2.0659, -0.0051,  ..., -0.3236, -0.1524,  0.9549]]])
torch.Size([1, 128, 768])


<ipython-input-15-caa1ad47fe50>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_embedding = torch.tensor(ds200['train'][1]["embeddings"])


In [ ]:
class FFNLayer(nn.Module):
    # T5LayerFF
    def __init__(self, config):
        super().__init__()
        self.linear1 = nn.Linear(config.d_model, config.d_ff)  # First projection
        self.activation = nn.GELU()  # Non-linear activation
        self.linear2 = nn.Linear(config.d_ff, config.d_model)  # Second projection
        self.dropout = nn.Dropout(config.dropout_rate)  # Dropout regularization
        self.layer_norm = nn.LayerNorm(config.d_model, eps=config.layer_norm_eps)  # Layer normalization

    def forward(self, hidden_states):

        #initializing weights in the feed-forward layers to improve stability during training.
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)

         # Input goes through the FFN
        forwarded_states = self.linear1(hidden_states)
        forwarded_states = self.activation(forwarded_states)
        forwarded_states = self.dropout(forwarded_states)
        forwarded_states = self.linear2(forwarded_states)

        # Add residual connection and layer norm
        hidden_states = self.layer_norm(hidden_states + forwarded_states)
        return hidden_states


In [ ]:
class AttentionLayer(nn.Module):
    # T5Attention
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_heads  # Number of attention heads
        self.d_model = config.d_model
        self.dropout = config.attention_dropout_rate

        self.self_attention = nn.MultiheadAttention(
            embed_dim=self.d_model,
            num_heads=self.num_heads,
            dropout=self.dropout
        )

        # skipping Relative positional bias

        self.layer_norm = nn.LayerNorm(self.d_model, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout_rate)

    def forward(self, hidden_states, attention_mask=None, layer_head_mask=None, output_attentions=False):
        """
        Forward pass for the attention layer.

        Args:
            hidden_states (torch.FloatTensor): Input tensor of shape [seq_len, batch_size, d_model].
            attention_mask (torch.FloatTensor, optional): Mask for attention mechanism of shape [batch_size, seq_len].
            layer_head_mask (torch.FloatTensor, optional): Mask for specific attention heads.
            output_attentions (bool, optional): Whether to return attention scores.

        Returns:
            torch.FloatTensor: Updated hidden states after attention.
            Optional[torch.FloatTensor]: Attention weights if `output_attentions=True`.
        """

        # Transpose hidden_states to [seq_len, batch_size, d_model] for nn.MultiheadAttention
        hidden_states = hidden_states.transpose(0, 1)  # [seq_len, batch_size, d_model]

        # Apply multi-head self-attention
        attention_output, attention_weights = self.self_attention(
            query=hidden_states,
            key=hidden_states,
            value=hidden_states,
            attn_mask=attention_mask
        )


        # Residual connection and layer normalization
        hidden_states = self.layer_norm(hidden_states + self.dropout(attention_output))

        # Transpose back to [batch_size, seq_len, d_model]
        hidden_states = hidden_states.transpose(0, 1)


        if output_attentions:
            return hidden_states, attention_weights
        return hidden_states

In [ ]:
class EncoderBlock(nn.Module):
   # a single Transformer encoder layer
   # similar to T5 Block
    def __init__(self, config):
        super(EncoderBlock, self).__init__()
        self.layer = nn.ModuleList()
        self.layer.append(AttentionLayer(config))
        self.layer.append(FFNLayer(config))


    def forward(
        self,
        hidden_states,
        attention_mask=None,
        # position_bias=None,
        layer_head_mask=None,
        #cross_attn_layer_head_mask=None,
        past_key_value=None,
        use_cache=False,
        output_attentions=False,
        return_dict=True,
        cache_position=None,
    ):
        """
        Forward pass for the encoder block.
        Args:
            hidden_states: Tensor of shape [?].
            attention_mask: Mask to prevent attention to certain positions.
            layer_head_mask: Mask for specific attention heads.
            use_cache: Whether to use caching for inference.
            output_attentions: Whether to return attention outputs.
            cache_position: Cache tracking the current position for inference.
        Returns:
            hidden_states: Updated hidden states after attention and FFN.
            outputs: A tuple containing additional optional outputs.
        """

        self_attention_outputs = self.layer[0](
            hidden_states,
            attention_mask=attention_mask,
           # layer_head_mask=layer_head_mask,
            #use_cache=use_cache,
            #output_attentions=output_attentions,
           # cache_position=cache_position,
        )
        if output_attentions:
            attention_outputs = self_attention_outputs[1:]
            hidden_states = self_attention_outputs[0]
        else:
            hidden_states = self_attention_outputs

        hidden_states = self.layer[1](hidden_states)

        # Clamp to handle FP16 training again
        if hidden_states.dtype == torch.float16:
            clamp_value = torch.finfo(hidden_states.dtype).max - 1000
            hidden_states = torch.clamp(hidden_states, min=-clamp_value, max=clamp_value)

        # Prepare outputs
        #outputs = (hidden_states,) + attention_outputs

        return hidden_states

# Initialize EncoderBlock
config = CustomConfig(d_model=768, num_heads=8, d_ff=2048)
encoder_block = EncoderBlock(config)

# Pass through EncoderBlock
block_output = encoder_block(hidden_states=train_embedding)
hidden_states= block_output  # Extract the hidden states
print(f"Output Shape from EncoderBlock: {hidden_states.shape}")  # Expected: [1, 1, 768]
print(f"Output from EncoderBlock: {hidden_states}")


Output Shape from EncoderBlock: torch.Size([1, 128, 768])
Output from EncoderBlock: tensor([[[-0.9104, -0.7036,  2.5679,  ..., -0.5355,  0.7111,  0.6737],
         [ 0.9147, -0.6334,  0.6842,  ...,  0.7939, -0.3808, -0.4310],
         [ 0.8379,  1.2476,  1.0183,  ...,  0.7513,  0.3041, -0.2136],
         ...,
         [ 0.1639, -0.9192,  1.3179,  ..., -0.1423, -0.3542,  1.2943],
         [ 0.2272, -1.7550, -0.1579,  ..., -0.0560, -0.5192,  1.4582],
         [-0.4959, -2.3812,  0.3081,  ..., -0.4283, -0.7118,  1.4269]]],
       grad_fn=<NativeLayerNormBackward0>)


In [ ]:
class TransformerEncoder(nn.Module):
  # stacks multiple encode layers
  # similar to T5Stack
    def __init__(self, config):
        super(TransformerEncoder, self).__init__()
        #self.embed_tokens = embed_tokens

        self.block = nn.ModuleList(
            [EncoderBlock(config) for i in range(config.num_encoder_layers)]
        )
        self.final_layer_norm = nn.LayerNorm(config.d_model, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout_rate)

        # Initialize weights and apply final processing
        # self.post_init()
        # # Model parallel
        # self.model_parallel = False
        # self.device_map = None
        # self.gradient_checkpointing = False



    def forward(
          self,
          hidden_states,
          attention_mask=None,
          head_mask=None,
          output_attentions=False,
          output_hidden_states=False,
          return_dict=True,
          ):
          """
          Forward pass for the TransformerEncoder.

          Args:
              hidden_states (Tensor): Input embeddings of shape [batch_size, seq_len, d_model].
              attention_mask (Tensor, optional): Mask to prevent attention to certain positions (e.g., padding).
              head_mask (Tensor, optional): Mask for specific attention heads.
              output_attentions (bool, optional): Whether to return attention weights.
              output_hidden_states (bool, optional): Whether to return intermediate hidden states.
              return_dict (bool, optional): Whether to return outputs as a dictionary.

          Returns:
              dict or tuple: Updated hidden states and optional outputs.
          """

          # Prepare outputs if needed
          all_hidden_states = () if output_hidden_states else None
          all_attentions = () if output_attentions else None

          # Apply dropout to input hidden states
          hidden_states = self.dropout(hidden_states)

          # Pass through each layer in the encoder block
          for i, layer_module in enumerate(self.block):
              if output_hidden_states:
                  all_hidden_states = all_hidden_states + (hidden_states,)

              # Forward pass through the encoder block
              print(hidden_states.shape)
              layer_outputs = layer_module(
                  hidden_states,
                  attention_mask=attention_mask,
                  layer_head_mask=head_mask[i] if head_mask is not None else None,
                  output_attentions=output_attentions,
              )

              # Collect attention weights if requested
              if output_attentions:
                  hidden_states = layer_outputs[0]
                  all_attentions = all_attentions + (layer_outputs[1],)
              else:
                  hidden_states = layer_outputs

          # Apply final layer normalization
          hidden_states = self.final_layer_norm(hidden_states)

          # Apply final dropout
          hidden_states = self.dropout(hidden_states)

          # Add the final hidden state to outputs if requested
          if output_hidden_states:
              all_hidden_states = all_hidden_states + (hidden_states,)

          # Prepare return values
          if not return_dict:
              return tuple(
                  v for v in [hidden_states, all_hidden_states, all_attentions] if v is not None
              )

          return {
              "last_hidden_state": hidden_states,
              "hidden_states": all_hidden_states,
              "attentions": all_attentions,
          }


# Initialize TransformerEncoder
config = CustomConfig(d_model=768, num_heads=8, d_ff=2048, num_encoder_layers=6)
transformer_encoder = TransformerEncoder(config)
# attention_mask = torch.ones(1, 1)
attention_mask = torch.ones(batch_embeddings.shape[1], batch_embeddings.shape[1])

# Pass through TransformerEncoder
encoder_output = transformer_encoder(hidden_states=train_embedding, attention_mask=attention_mask)

# Print the last hidden state
print(f"Output Shape from TransformerEncoder: {encoder_output['last_hidden_state'].shape}")  # Expected: [1, 1, 768]
print(f"Last Hidden State: {encoder_output['last_hidden_state']}")



torch.Size([1, 128, 768])
torch.Size([1, 128, 768])
torch.Size([1, 128, 768])
torch.Size([1, 128, 768])
torch.Size([1, 128, 768])
torch.Size([1, 128, 768])
Output Shape from TransformerEncoder: torch.Size([1, 128, 768])
Last Hidden State: tensor([[[ 0.4964, -0.1527,  1.6809,  ...,  0.4266,  2.4336, -1.1814],
         [ 0.4721, -0.0000,  1.3222,  ...,  0.5020,  0.5195, -0.9262],
         [ 0.5314,  0.7026,  0.9468,  ...,  1.2461,  1.9881,  0.2304],
         ...,
         [ 0.1144,  0.1163,  0.5316,  ...,  0.7115,  3.2617,  1.1738],
         [ 1.4265, -0.3328,  1.5728,  ...,  0.8218,  2.7087,  2.2034],
         [ 1.0433,  0.1932,  1.3897,  ...,  2.0250,  2.0239,  1.5323]]],
       grad_fn=<MulBackward0>)


In [ ]:
class EncoderModel(nn.Module):
  # wraps the transformer encoder
  # similar to T5EncoderModel


    def __init__(self, config):
      super(EncoderModel, self).__init__()
      # Embedding layer (token + positional)
      # self.token_embedding = nn.Embedding(config., hidden_size)
      # self.position_embedding = nn.Embedding(max_seq_len, hidden_size)
      # self.dropout = nn.Dropout(dropout)
      self.encoder = TransformerEncoder(config)
      # self.hidden_size = config.h


    def forward(
        self,
        hidden_states: torch.FloatTensor,  # Precomputed embeddings
        attention_mask = None,
        output_attentions = False,
        output_hidden_states = False,
        return_dict = True,
    ) -> Union[Tuple[torch.FloatTensor], dict]:
      """
      Forward pass for the EncoderModel.

      Args:
          hidden_states (torch.FloatTensor): Precomputed embeddings of shape [batch_size, seq_len, d_model].
          attention_mask (Optional[torch.FloatTensor]): Mask of shape [batch_size, seq_len] to prevent attention to padding tokens.
          output_attentions (Optional[bool]): Whether to return attention weights.
          output_hidden_states (Optional[bool]): Whether to return hidden states of all layers.
          return_dict (Optional[bool]): Whether to return outputs as a dictionary.

      Returns:
          Union[Tuple, dict]: Final hidden states and optional outputs (attention weights, hidden states).
      """

      # Forward pass through the transformer encoder
      print(hidden_states.shape)
      encoder_outputs = self.encoder(
          hidden_states=hidden_states,
          attention_mask=attention_mask,
          output_attentions=output_attentions,
          output_hidden_states=output_hidden_states,
          return_dict=return_dict,
      )

      return encoder_outputs

      # if not return_dict:
      #     return (hidden_states, all_hidden_states, all_attentions)
      # return {
      #     "last_hidden_state": hidden_states,
      #     "hidden_states": all_hidden_states,
      #     "attentions": all_attentions,
      # }




    # def forward(self, input_ids, attention_mask=None):



    #   return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    #   encoder_outputs = self.encoder(
    #         input_ids=input_ids,
    #         attention_mask=attention_mask,
    #         inputs_embeds=inputs_embeds,
    #         head_mask=head_mask,
    #         output_attentions=output_attentions,
    #         output_hidden_states=output_hidden_states,
    #         return_dict=return_dict,
    #     )

    #   return encoder_outputs


      # seq_len = input_ids.size(1)
      # # Create position IDs
      # position_ids = torch.arange(seq_len, dtype=torch.long, device=input_ids.device)
      # position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

      # # Combine token and position embeddings
      # embeddings = self.token_embedding(input_ids) + self.position_embedding(position_ids)
      # embeddings = self.dropout(embeddings)

      # # Pass through TransformerEncoder
      # encoded_output = self.encoder(embeddings.transpose(0, 1), attention_mask)  # Transpose for attention

      # return encoded_output.transpose(0, 1)  # Transpose back to original format

    # def parallelize(self, device_map=None):
    #   warnings.warn(
    #       "`T5EncoderModel.parallelize` is deprecated and will be removed in v5 of Transformers, you should load"
    #       " your model with `device_map='balanced'` in the call to `from_pretrained`. You can also provide your own"
    #       " `device_map` but it needs to be a dictionary module_name to device, so for instance {'block.0': 0,"
    #       " 'block.1': 1, ...}",
    #       FutureWarning,
    #   )
    #   self.device_map = (
    #       get_device_map(len(self.encoder.block), range(torch.cuda.device_count()))
    #       if device_map is None
    #       else device_map
    #   )
    #   assert_device_map(self.device_map, len(self.encoder.block))
    #   self.encoder.parallelize(self.device_map)
    #   self.model_parallel = True

    # @add_start_docstrings(DEPARALLELIZE_DOCSTRING)
    # def deparallelize(self):
    #   warnings.warn(
    #       "Like `parallelize`, `deparallelize` is deprecated and will be removed in v5 of Transformers.",
    #       FutureWarning,
    #   )
    #   self.encoder.deparallelize()
    #   self.encoder = self.encoder.to("cpu")
    #   self.model_parallel = False
    #   self.device_map = None
    #   torch.cuda.empty_cache()

    # def get_input_embeddings(self):
    #   return self.shared

    # def set_input_embeddings(self, new_embeddings):
    #   self.shared = new_embeddings
    #   self.encoder.set_input_embeddings(new_embeddings)

    # def _tie_weights(self):
    #   if self.config.tie_word_embeddings:
    #       self._tie_or_clone_weights(self.encoder.embed_tokens, self.shared)


In [ ]:
config = CustomConfig(d_model=768, num_heads=8, d_ff=2048, num_encoder_layers=6)
encoder_model = EncoderModel(config)

# Pass the embedding through the encoder
output = encoder_model(batch_embeddings)
print(output)

torch.Size([200, 128, 768])
torch.Size([200, 128, 768])
torch.Size([200, 128, 768])
torch.Size([200, 128, 768])
torch.Size([200, 128, 768])
torch.Size([200, 128, 768])
torch.Size([200, 128, 768])


# Decoder



In [ ]:
class CrossAttentionLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_heads
        self.d_model = config.d_model
        self.dropout = config.attention_dropout_rate

        self.cross_attention = nn.MultiheadAttention(
            embed_dim=self.d_model,
            num_heads=self.num_heads,
            dropout=self.dropout
        )

        self.layer_norm = nn.LayerNorm(self.d_model, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout_rate)

    def forward(self, hidden_states, encoder_hidden_states, attention_mask=None):
        hidden_states = hidden_states.transpose(0, 1)
        encoder_hidden_states = encoder_hidden_states.transpose(0, 1)

        attention_output, attention_weights = self.cross_attention(
            query=hidden_states,
            key=encoder_hidden_states,
            value=encoder_hidden_states,
            attn_mask=attention_mask
        )

        hidden_states = self.layer_norm(hidden_states + self.dropout(attention_output))
        hidden_states = hidden_states.transpose(0, 1)

        return hidden_states

class DecoderBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self_attention = AttentionLayer(config)
        self.cross_attention = CrossAttentionLayer(config)
        self.ffn = FFNLayer(config)

    def forward(
        self,
        hidden_states,
        encoder_hidden_states,
        attention_mask=None,
        encoder_attention_mask=None,
        layer_head_mask=None,
        cross_attn_layer_head_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        # Self Attention
        self_attention_outputs = self.self_attention(
            hidden_states,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            output_attentions=output_attentions
        )
        hidden_states = self_attention_outputs if not output_attentions else self_attention_outputs[0]

        # Cross Attention
        cross_attention_outputs = self.cross_attention(
            hidden_states,
            encoder_hidden_states,
            attention_mask=encoder_attention_mask
        )
        hidden_states = cross_attention_outputs

        # Feed Forward
        hidden_states = self.ffn(hidden_states)

        return hidden_states

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.block = nn.ModuleList(
            [DecoderBlock(config) for _ in range(config.num_decoder_layers)]
        )
        self.final_layer_norm = nn.LayerNorm(config.d_model, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.dropout_rate)
        self.output_projection = nn.Linear(config.d_model, config.vocab_size, bias=False)

    def forward(
        self,
        hidden_states,
        encoder_hidden_states,
        attention_mask=None,
        encoder_attention_mask=None,
        head_mask=None,
        cross_attn_head_mask=None,
        past_key_values=None,
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True,
    ):
        all_hidden_states = () if output_hidden_states else None
        all_self_attns = () if output_attentions else None
        all_cross_attns = () if output_attentions else None

        hidden_states = self.dropout(hidden_states)

        for i, layer_module in enumerate(self.block):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_head_mask = head_mask[i] if head_mask is not None else None
            cross_attn_layer_head_mask = (
                cross_attn_head_mask[i] if cross_attn_head_mask is not None else None
            )

            hidden_states = layer_module(
                hidden_states,
                encoder_hidden_states,
                attention_mask=attention_mask,
                encoder_attention_mask=encoder_attention_mask,
                layer_head_mask=layer_head_mask,
                cross_attn_layer_head_mask=cross_attn_layer_head_mask,
                output_attentions=output_attentions,
            )

        # Final layer norm
        hidden_states = self.final_layer_norm(hidden_states)
        hidden_states = self.dropout(hidden_states)

        # Project to vocabulary
        logits = self.output_projection(hidden_states)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(v for v in [
                logits,
                all_hidden_states,
                all_self_attns,
                all_cross_attns
            ] if v is not None)

        return {
            "logits": logits,
            "hidden_states": all_hidden_states,
            "self_attentions": all_self_attns,
            "cross_attentions": all_cross_attns,
        }

In [ ]:
class T5Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.decoder = TransformerDecoder(config)
        self.config = config

    def forward(
        self,
        encoder_input,
        decoder_input,
        attention_mask=None,
        decoder_attention_mask=None,
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True,
    ):
        # Encode
        encoder_outputs = self.encoder(
            hidden_states=encoder_input,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # Decode
        decoder_outputs = self.decoder(
            hidden_states=decoder_input,
            encoder_hidden_states=encoder_outputs["last_hidden_state"],
            attention_mask=decoder_attention_mask,
            encoder_attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        if not return_dict:
            return (decoder_outputs["logits"],)

        return {
            "logits": decoder_outputs["logits"],
            "encoder_hidden_states": encoder_outputs["hidden_states"],
            "decoder_hidden_states": decoder_outputs["hidden_states"],
            "encoder_attentions": encoder_outputs["attentions"],
            "decoder_attentions": decoder_outputs["self_attentions"],
            "cross_attentions": decoder_outputs["cross_attentions"],
        }

In [ ]:
# def create_sample_config():
#     return CustomConfig(
#         d_model=768,
#         num_heads=8,
#         d_ff=2048,
#         num_encoder_layers=6,
#         num_decoder_layers=6,
#         vocab_size=32128,  # Example vocabulary size
#         dropout_rate=0.1,
#         attention_dropout_rate=0.1,
#         layer_norm_eps=1e-6
#     )

# # Initialize model
# config = create_sample_config()
# model = T5Model(config)

# # Create sample inputs
# batch_size = 2
# seq_length = 10
# encoder_input = torch.randn(batch_size, seq_length, config.d_model)
# decoder_input = torch.randn(batch_size, seq_length, config.d_model)
# attention_mask = torch.ones(batch_size, seq_length)
# decoder_attention_mask = torch.ones(batch_size, seq_length)

# # Forward pass
# outputs = model(
#     encoder_input=encoder_input,
#     decoder_input=decoder_input,
#     attention_mask=attention_mask,
#     decoder_attention_mask=decoder_attention_mask,
#     output_attentions=True,
#     output_hidden_states=True,
# )

# # Print output shapes
# print("Logits shape:", outputs["logits"].shape)
# print("Encoder hidden states shape:",
#       [hidden.shape for hidden in outputs["encoder_hidden_states"]] if outputs["encoder_hidden_states"] else None)
# print("Decoder hidden states shape:",
#       [hidden.shape for hidden in outputs["decoder_hidden_states"]] if outputs["decoder_hidden_states"] else None)

config = CustomConfig(d_model=768, num_heads=8, d_ff=2048,
                         num_encoder_layers=6, num_decoder_layers=6)

    # Initialize decoder
    decoder = TransformerDecoder(config)

    # Create sample inputs
    decoder_input = batch_embeddings
    encoder_hidden_states = batch_embeddings  # Simulating encoder output

    # Create attention masks
    attention_mask = torch.ones(batch_embeddings.size(0), batch_embeddings.size(1))
    encoder_attention_mask = torch.ones(batch_embeddings.size(0), batch_embeddings.size(1))

    # Forward pass
    outputs = decoder(
        hidden_states=decoder_input,
        encoder_hidden_states=encoder_hidden_states,
        attention_mask=attention_mask,
        encoder_attention_mask=encoder_attention_mask,
    )

    print("Decoder input shape:", decoder_input.shape)

# DataLoader

In [ ]:
from torch.utils.data import Sampler, Dataset
from collections import OrderedDict
from random import shuffle


class BucketDataset(Dataset):

    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        if self.targets is None:
            return self.inputs[index]
        else:
            return self.inputs[index], self.targets[index]


class BucketBatchSampler(Sampler):
    # want inputs to be an array
    def __init__(self, inputs, batch_size):
        self.batch_size = batch_size
        ind_n_len = []
        for i, p in enumerate(inputs):
            ind_n_len.append((i, p.shape[0]))
        self.ind_n_len = ind_n_len
        self.batch_list = self._generate_batch_map()
        self.num_batches = len(self.batch_list)

    def _generate_batch_map(self):
        # shuffle all of the indices first so they are put into buckets differently
        shuffle(self.ind_n_len)
        # Organize lengths, e.g., batch_map[10] = [30, 124, 203, ...] <= indices of sequences of length 10
        batch_map = OrderedDict()
        for idx, length in self.ind_n_len:
            if length not in batch_map:
                batch_map[length] = [idx]
            else:
                batch_map[length].append(idx)
        # Use batch_map to split indices into batches of equal size
        # e.g., for batch_size=3, batch_list = [[23,45,47], [49,50,62], [63,65,66], ...]
        batch_list = []
        for length, indices in batch_map.items():
            for group in [indices[i:(i + self.batch_size)] for i in range(0, len(indices), self.batch_size)]:
                batch_list.append(group)
        return batch_list

    def batch_count(self):
        return self.num_batches

    def __len__(self):
        return len(self.ind_n_len)

    def __iter__(self):
        self.batch_list = self._generate_batch_map()
        # shuffle all the batches so they arent ordered by bucket size
        shuffle(self.batch_list)
        for i in self.batch_list:
            yield i

In [ ]:
sampler = BucketDataset(<your data in an np.array>, BATCH_SIZE)
dataloader = DataLoader(<your data as a DataSet object>, batch_size=1,
                            batch_sampler=sampler, shuffle=False,
                            num_workers=8,  drop_last=False)
